In [1]:
!which python

/u/pw7nc/anaconda3/bin/python


In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

In [3]:
# Create a new model
m = gp.Model("graph2x_ilp")
m.Params.LogToConsole = 0

In [4]:
# Create variables
X = m.addMVar(shape=5, vtype=GRB.BINARY, name="X")

In [5]:
Y = m.addMVar(shape=(5,5), vtype=GRB.BINARY, name="Y")

In [6]:
X

<(5,) matrix variable>

In [7]:
Y

<(5, 5) matrix variable>

In [8]:
X[0]

<(1,) matrix variable>

In [9]:
Y[0][0]

<(1,) matrix variable>

In [10]:
m

<gurobi.Model Continuous instance graph2x_ilp: 0 constrs, 0 vars, Parameter changes: LogToConsole=0>

In [11]:
sim_scores = np.array(
    [
        [1.0, 0.16, 0.2, 0.3, 0.5],
        [0.16, 1.0, 0.7, 0.4, 0.2],
        [0.2, 0.7, 1.0, 0.9, 0.25],
        [0.3, 0.4, 0.9, 1.0, 0.12],
        [0.5, 0.2, 0.25, 0.12, 1.0]
    ]
)

In [12]:
sim_scores

array([[1.  , 0.16, 0.2 , 0.3 , 0.5 ],
       [0.16, 1.  , 0.7 , 0.4 , 0.2 ],
       [0.2 , 0.7 , 1.  , 0.9 , 0.25],
       [0.3 , 0.4 , 0.9 , 1.  , 0.12],
       [0.5 , 0.2 , 0.25, 0.12, 1.  ]])

In [13]:
sim_scores_upper = np.triu(sim_scores, 1)

In [14]:
sim_scores_upper

array([[0.  , 0.16, 0.2 , 0.3 , 0.5 ],
       [0.  , 0.  , 0.7 , 0.4 , 0.2 ],
       [0.  , 0.  , 0.  , 0.9 , 0.25],
       [0.  , 0.  , 0.  , 0.  , 0.12],
       [0.  , 0.  , 0.  , 0.  , 0.  ]])

In [15]:
pred_scores = np.array(
    [0.8, 0.6, 0.5, 0.2, 0.01]
)

In [16]:
pred_scores

array([0.8 , 0.6 , 0.5 , 0.2 , 0.01])

In [17]:
alpha = 0.5

# Option 1: Use List Comprehension
m.setObjective(
    pred_scores @ X - alpha * sum(Y[i][j] * sim_scores[i][j] for i in range(5) for j in range(i+1, 5)),
    GRB.MAXIMIZE
)

# # Option 2: Use Matrix matmul. Note, the sim_score should only contain the upper triangle matrix
# m.setObjective(
#     (pred_scores @ X) - alpha * (X @ sim_scores_upper @ X),
#     GRB.MAXIMIZE
# )

In [18]:
np.ones(5)

array([1., 1., 1., 1., 1.])

In [19]:
# In this toy example, let's try to select 3 sentences
N_num = 3
ones = np.ones(5)
m.addConstr(ones @ X == N_num, name="c")

<(1,) matrix constraint>

In [20]:
# Add the inequality constraints
# https://www.gurobi.com/documentation/9.1/refman/py_model_addconstrs.html
m.addConstrs(
    ((X[i] + X[j]) <= (Y[i][j] + 1) for i in range(5) for j in range(i+1, 5)), name='c1'
)

{(0, 1): <(1,) matrix constraint *awaiting model update*>,
 (0, 2): <(1,) matrix constraint *awaiting model update*>,
 (0, 3): <(1,) matrix constraint *awaiting model update*>,
 (0, 4): <(1,) matrix constraint *awaiting model update*>,
 (1, 2): <(1,) matrix constraint *awaiting model update*>,
 (1, 3): <(1,) matrix constraint *awaiting model update*>,
 (1, 4): <(1,) matrix constraint *awaiting model update*>,
 (2, 3): <(1,) matrix constraint *awaiting model update*>,
 (2, 4): <(1,) matrix constraint *awaiting model update*>,
 (3, 4): <(1,) matrix constraint *awaiting model update*>}

In [21]:
# Add the sum constraint of Y
E_num = N_num * (N_num - 1) / 2
m.addConstr(
    sum(Y[i][j] for i in range(5) for j in range(i+1, 5)) == E_num, name="c2"
)

<(1,) matrix constraint *awaiting model update*>

In [22]:
m

<gurobi.Model Continuous instance graph2x_ilp: 0 constrs, 0 vars, Parameter changes: LogToConsole=0>

In [23]:
# Optimize model
m.optimize()

In [24]:
m

<gurobi.Model MIP instance graph2x_ilp: 12 constrs, 30 vars, Parameter changes: LogToConsole=0>

In [25]:
print(X.X)
print(Y.X)
print('Obj: %g' % m.objVal)

[ 1.  1.  1. -0. -0.]
[[ 0.  1.  1.  0.  0.]
 [ 0.  0.  1. -0. -0.]
 [ 0.  0.  0. -0. -0.]
 [ 0.  0.  0.  0. -0.]
 [ 0.  0.  0.  0.  0.]]
Obj: 1.37


In [26]:
m

<gurobi.Model MIP instance graph2x_ilp: 12 constrs, 30 vars, Parameter changes: LogToConsole=0>

In [52]:
m.dispose()

In [27]:
Y.X

array([[ 0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  1., -0., -0.],
       [ 0.,  0.,  0., -0., -0.],
       [ 0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [28]:
Y.X[0][1] == 1.0

True

In [29]:
Y.X[1][3] == 0.0

True

In [30]:
Y.X[1][4] == 0.0

True